In [3]:
#from google.colab import files
#uploaded = files.upload()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle 
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [7]:
def my_read(X):
  pre_data = pq.read_table(X)
  data = pre_data.to_pandas()
  return data

#Pre_Jan = pq.read_table("/fhv_tripdata_2021-01.parquet")
#Jan = Pre_Jan.to_pandas()
#Jan.head()

In [9]:
January = my_read("/fhv_tripdata_2021-01.parquet")
February = my_read("/fhv_tripdata_2021-02.parquet")

In [10]:
January.shape, February.shape

((1154112, 7), (1037692, 7))

In [51]:
January.sample(5)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,Jan_Duration
350803,B01285,2021-01-11 08:27:34,2021-01-11 08:40:07,NaN,47.0,None,B01285,12.550000
898803,B01083,2021-01-25 12:24:48,2021-01-25 12:52:37,NaN,244.0,None,B01083,27.816667
626644,B00412,2021-01-18 11:43:18,2021-01-18 11:48:52,NaN,NaN,None,B01899,5.566667
67728,B00310,2021-01-03 15:06:25,2021-01-03 15:20:00,NaN,250.0,None,B02880,13.583333
128703,B01509,2021-01-05 09:23:05,2021-01-05 09:27:50,NaN,29.0,None,B01509,4.750000


In [52]:
January.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
Jan_Duration                     float64
dtype: object

In [54]:
January["Jan_Duration"] = January.dropOff_datetime - January.pickup_datetime

In [15]:
January.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,Jan_Duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,0 days 00:17:00
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,0 days 00:17:00
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,0 days 01:50:00
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,0 days 00:08:17
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,0 days 00:15:13


In [55]:
January.describe()

,PUlocationID,DOlocationID,Jan_Duration
count,195845.000000,991892.000000,1154112
mean,139.859690,135.898030,0 days 00:19:10.033445627
std,74.991382,80.474902,0 days 06:38:41.529882844
min,1.000000,1.000000,0 days 00:00:01
25%,75.000000,67.000000,0 days 00:07:46
50%,143.000000,132.000000,0 days 00:13:24
75%,206.000000,213.000000,0 days 00:22:17
max,265.000000,265.000000,294 days 00:11:03


In [56]:
January.Jan_Duration = January.Jan_Duration.apply(lambda td: td.total_seconds() / 60)

In [25]:
mean_jan_duration = January.Jan_Duration.mean()
mean_jan_duration

19.1672240937939

In [59]:
Jan = January[(January.Jan_Duration >= 1) & (January.Jan_Duration <= 60)]
Jan

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,Jan_Duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.050000
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.433333


In [61]:
Jan.shape

(1109826, 8)

In [60]:
Jan.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,Jan_Duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.050000


In [62]:
len(January) - len(Jan)

44286

In [27]:
January.isna().sum()

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID               958267
DOlocationID               162220
SR_Flag                   1154112
Affiliated_base_number        885
Jan_Duration                    0
dtype: int64

In [63]:
Jan.isna().sum()

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID               927008
DOlocationID               147907
SR_Flag                   1109826
Affiliated_base_number        773
Jan_Duration                    0
dtype: int64

In [64]:
Jan["PUlocationID"].isna().sum()

927008

In [67]:
Jan["PUlocationID"].fillna(-1, inplace=True)
Jan["DOlocationID"].fillna(-1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [68]:
Jan.isna().sum()

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID                    0
DOlocationID                    0
SR_Flag                   1109826
Affiliated_base_number        773
Jan_Duration                    0
dtype: int64

In [78]:
Filled = Jan[(Jan["PUlocationID"] == -1)]
Filled

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,Jan_Duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,9.050000
...,...,...,...,...,...,...,...,...
1154094,B03202,2021-01-31 23:57:02,2021-02-01 00:14:27,-1.0,177.0,None,B03202,17.416667
1154096,B03239,2021-01-31 23:07:00,2021-01-31 23:17:28,-1.0,70.0,None,B03239,10.466667
1154097,B03239,2021-01-31 23:27:39,2021-01-31 23:33:38,-1.0,82.0,None,B03239,5.983333
1154098,B03239,2021-01-31 23:40:10,2021-01-31 23:52:07,-1.0,56.0,None,B03239,11.950000


In [80]:
(len(Filled) / len(Jan)) * 100

83.52732770722618

In [81]:
Jan.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
Jan_Duration                     float64
dtype: object

In [100]:
categorical = ["PUlocationID", "DOlocationID"]

In [91]:
Jan.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number',
       'Jan_Duration'],
      dtype='object')

In [101]:
categorical

['PUlocationID', 'DOlocationID']

In [102]:
#Jan[categorical].values

In [103]:
Jan[categorical] = Jan[categorical].astype(str)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [107]:
train_dicts = Jan[categorical].to_dict(orient="records")

In [108]:
#train_dicts

[{'DOlocationID': '-1.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '-1.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '72.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '61.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '71.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '91.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '39.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '37.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '39.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '72.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '72.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '89.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '177.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '225.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '63.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '67.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '22.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '61.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '14.0', 'PUlocationID': '-1.0'},
 {'DOlocationID': '14.0', 'PU

In [111]:
dv = DictVectorizer()
X = dv.fit_transform(train_dicts)

target = "Jan_Duration"
Y = Jan[target].values

In [126]:
X.shape

(1109826, 525)

In [121]:
from sklearn.model_selection import train_test_split
import math

In [113]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=25)
# = train_test_split(Y, test_size=0.2, random_state=25)

In [116]:
print("Train set:", X_train.shape, y_train.shape)
print("Val set:", X_val.shape, y_val.shape)

Train set: (887860, 525) (887860,)
Val set: (221966, 525) (221966,)


In [117]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [118]:
y_pred = lr.predict(X_train)

In [123]:
MSE = mean_squared_error(y_train, y_pred, squared=False)
print(MSE)
RMSE = math.sqrt(MSE)
print(RMSE)

10.528871756025863
3.2448222996068465


In [124]:
y_val_pred = lr.predict(X_val)

In [125]:
MSE = mean_squared_error(y_val, y_val_pred, squared=False)
print(MSE)
RMSE = math.sqrt(MSE)
print(RMSE)

10.531109643924749
3.2451671211086723


In [12]:
February.sample(5)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
3605,B00837,2021-02-01 08:10:23,2021-02-01 08:20:48,NaN,NaN,None,B00837
794562,B02674,2021-02-22 18:50:57,2021-02-22 19:12:06,102.0,35.0,None,B02674
315930,B01051,2021-02-10 09:12:47,2021-02-10 09:15:55,NaN,126.0,None,B01051
278555,B03047,2021-02-09 10:30:38,2021-02-09 10:41:20,242.0,254.0,None,B02534
53915,B02803,2021-02-03 11:40:18,2021-02-03 11:44:35,247.0,47.0,None,B02803
